In [1]:
import json
args = (str("--output_dir /mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/toy --model_name_or_path /mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/saved_model_v100 --train_data /opt/tiger/toy_data.jsonl --learning_rate 1e-5 --fp16 --num_train_epochs 3 --per_device_train_batch_size 4 --gradient_accumulation_steps 4 --dataloader_drop_last --train_group_size 16 --max_len 512 --weight_decay 0.01 --logging_steps 10 --save_strategy epoch --save_steps 1 --save_total_limit 3".split(" ")).replace("\'","\""))
print(args)

["--output_dir", "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/toy", "--model_name_or_path", "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/saved_model_v100", "--train_data", "/opt/tiger/toy_data.jsonl", "--learning_rate", "1e-5", "--fp16", "--num_train_epochs", "3", "--per_device_train_batch_size", "4", "--gradient_accumulation_steps", "4", "--dataloader_drop_last", "--train_group_size", "16", "--max_len", "512", "--weight_decay", "0.01", "--logging_steps", "10", "--save_strategy", "epoch", "--save_steps", "1", "--save_total_limit", "3"]
